#### 추천시스템(Recommend System)
* 추천 시스템은 크게 두 가지로 구분 가능
    - 컨텐츠 기반 필터링
    - 협업 필터링
* 두 가지를 조합한 hybrid 방식 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 

#### Surprise
* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
